In [1]:
import pandas as pd
from google.colab import files
import io

In [2]:
uploaded = files.upload()

Saving traindata.csv to traindata.csv


In [3]:
data = io.BytesIO(uploaded["traindata.csv"])
df = pd.read_csv(data, sep = None, engine = 'python')
df

,id,x1,x2,x3,y
0,1,60,64,0,1
1,2,54,60,11,0
2,3,65,62,22,0
3,4,34,60,0,1
4,5,38,69,21,0
...,...,...,...,...,...
292,293,65,67,0,1
293,294,53,65,12,0
294,295,57,64,1,0
295,296,54,59,7,1


In [4]:
df.describe()

,id,x1,x2,x3,y
count,297.000000,297.000000,297.000000,297.000000,297.000000
mean,148.996633,52.468013,62.868687,4.121212,0.737374
std,85.874929,10.878311,3.236134,7.281418,0.440804
min,1.000000,30.000000,58.000000,0.000000,0.000000
25%,75.000000,44.000000,60.000000,0.000000,0.000000
50%,149.000000,52.000000,63.000000,1.000000,1.000000
75%,223.000000,61.000000,65.000000,5.000000,1.000000
max,296.000000,83.000000,69.000000,52.000000,1.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      297 non-null    int64
 1   x1      297 non-null    int64
 2   x2      297 non-null    int64
 3   x3      297 non-null    int64
 4   y       297 non-null    int64
dtypes: int64(5)
memory usage: 11.7 KB


#Data Preprocessing


In [6]:
#Split data untuk memisahkan features data dan target data
#data fitur
features = df[['x1', 'x2', 'x3']]
#data target
target = df["y"]

features.head()

,x1,x2,x3
0,60,64,0
1,54,60,11
2,65,62,22
3,34,60,0
4,38,69,21


#Folding
memisahkan dataframe menjadi beberapa bagian untuk melihat karakteristik data

In [7]:
#mengambil quarter pertama untuk testing dan 3 quarter sisanya untuk training
fold_first_quarter = (df.iloc[0:74].reset_index(drop=True),              df.iloc[74:296].reset_index(drop=True))

#mengambil 50 persen data dengan menyilang masing-masing untuk training dan testing
fold_first_half = (pd.concat([df.iloc[0:74],df.iloc[148:222]]).reset_index(drop=True), pd.concat([df.iloc[74:148],df.iloc[222:296]]).reset_index(drop=True))
fold_second_half = (pd.concat([df.iloc[74:148],df.iloc[148:222]]).reset_index(drop=True), pd.concat([df.iloc[0:74],df.iloc[222:296]]).reset_index(drop=True))

#mengambil quarter akhir untuk testing dan quarter awal hingga quarter ke-tiga untuk training
fold_last_quarter = (df.iloc[0:222].reset_index(drop=True),           df.iloc[222:296].reset_index(drop=True))

fold_first_quarter

(    id  x1  x2  x3  y
 0    1  60  64   0  1
 1    2  54  60  11  0
 2    3  65  62  22  0
 3    4  34  60   0  1
 4    5  38  69  21  0
 ..  ..  ..  ..  .. ..
 69  70  64  66   0  1
 70  71  37  63   0  1
 71  72  62  62   6  1
 72  73  38  64   1  1
 73  74  63  62   0  1
 
 [74 rows x 5 columns],
       id  x1  x2  x3  y
 0     75  55  66   0  1
 1     76  61  68   0  1
 2     77  50  59   0  1
 3     78  43  58  52  0
 4     79  78  65   1  0
 ..   ...  ..  ..  .. ..
 217  292  59  64   1  1
 218  293  65  67   0  1
 219  294  53  65  12  0
 220  295  57  64   1  0
 221  296  54  59   7  1
 
 [222 rows x 5 columns])

In [8]:
features.iloc[0] - features.iloc[1]

x1     6
x2     4
x3   -11
dtype: int64

#Function Distance
2 rumus yang dapat dipakai untuk menghitung jarak

In [9]:
def euclidean_distance(x, y):
    dist = 0
    for i in range(len(y)):
        dist += (x[i] - y[i])**2

    return dist ** (1/2)

def manhattan_distance(x, y):
    return (x-y).abs().sum()

In [10]:
print(manhattan_distance(features.iloc[0], features.iloc[1]))
print(euclidean_distance(features.iloc[0], features.iloc[1]))

21
13.152946437965905


#Normalisasi
###mengubah jarak data dari 0-1 agar jarak tidak terlalu timpang


In [11]:
features = (features - features.min()) / (features.max()-features.min())

In [12]:
features.describe()

,x1,x2,x3
count,297.000000,297.000000,297.000000
mean,0.423925,0.442608,0.079254
std,0.205251,0.294194,0.140027
min,0.000000,0.000000,0.000000
25%,0.264151,0.181818,0.000000
50%,0.415094,0.454545,0.019231
75%,0.584906,0.636364,0.096154
max,1.000000,1.000000,1.000000


In [13]:
def predict(x,k,X,y):
  distance = []
  for _, row in X.iterrows():
    distance.append(manhattan_distance(x,row))

  data = X.copy()
  data['distance'] = distance
  data['clas'] = y
  data.sort_values(by="distance").reset_index(drop=True)

  y_pred = data.iloc[:k].clas.mode()

  return y_pred[0]



In [14]:
predict(features.iloc[6],1, features, target)

1

In [15]:
def akurasi(y_pred,y_true):
    n = len(y_pred)
    benar = 0
    for i in range(n):
       if y_pred[i] == y_true[i]:
        benar += 1
    return benar /n * 100

In [16]:
akurasi(target, target)

100.0

In [17]:
accurate = []

for k in range(1,100):
    for fold in [fold_first_quarter, fold_first_half, fold_second_half, fold_last_quarter]:
        train, test = fold
        X_train, y_train = train.drop('y', axis=1), train.y
        X_test, y_test = test.drop('y', axis=1), test.y

        y_preds = []
        for _, x in X_test.iterrows():
            y_preds.append(predict(x, k, X_train, y_train))

        accurate.append(akurasi(y_preds, y_test))

    print(f"Accuracy for k={k}: {sum(accurate) / len(accurate):.2f}%")

Accuracy for k=1: 72.58%
Accuracy for k=2: 55.57%
Accuracy for k=3: 49.91%
Accuracy for k=4: 47.07%
Accuracy for k=5: 45.37%
Accuracy for k=6: 44.24%
Accuracy for k=7: 48.29%
Accuracy for k=8: 51.32%
Accuracy for k=9: 53.68%
Accuracy for k=10: 55.57%
Accuracy for k=11: 57.12%
Accuracy for k=12: 58.41%
Accuracy for k=13: 59.50%
Accuracy for k=14: 60.43%
Accuracy for k=15: 61.24%
Accuracy for k=16: 61.95%
Accuracy for k=17: 62.58%
Accuracy for k=18: 63.13%
Accuracy for k=19: 63.63%
Accuracy for k=20: 64.08%
Accuracy for k=21: 64.48%
Accuracy for k=22: 64.85%
Accuracy for k=23: 65.19%
Accuracy for k=24: 65.49%
Accuracy for k=25: 65.78%
Accuracy for k=26: 66.04%
Accuracy for k=27: 66.28%
Accuracy for k=28: 66.51%
Accuracy for k=29: 66.72%
Accuracy for k=30: 66.91%
Accuracy for k=31: 67.09%
Accuracy for k=32: 67.26%
Accuracy for k=33: 67.43%
Accuracy for k=34: 67.58%
Accuracy for k=35: 67.72%
Accuracy for k=36: 67.86%
Accuracy for k=37: 67.98%
Accuracy for k=38: 68.10%
Accuracy for k=39: 68